In [22]:
import pandas as pd

In [23]:
data = pd.read_csv('dataset.csv')

In [24]:
print(data.head())
print(data.info())
print(data.describe())

                        date   open    high     low  close  volume    sma5  \
0  2015-02-02 14:30:00+05:30  336.5  336.50  335.70  335.8    7874  336.31   
1  2015-02-02 14:35:00+05:30  335.8  335.80  335.00  335.3   16845  336.09   
2  2015-02-02 14:40:00+05:30  335.3  335.95  335.05  335.2   12125  335.85   
3  2015-02-02 14:45:00+05:30  335.2  335.20  333.55  334.8   43527  335.47   
4  2015-02-02 14:50:00+05:30  334.8  336.00  334.25  335.5   25387  335.32   

     sma10       sma15     sma20  ...      fastd  fastksr    fastdsr  \
0  336.465  336.503333  336.6000  ...  23.551757      0.0  22.002387   
1  336.325  336.470000  336.5050  ...  12.502374      0.0   0.000000   
2  336.170  336.383333  336.3800  ...  11.072874      0.0   0.000000   
3  336.020  336.240000  336.2700  ...  21.329285      0.0   0.000000   
4  335.870  336.133333  336.2175  ...  38.363183    100.0  33.333333   

      ULTOSC      WILLR       ATR  Trange    TYPPRICE  HT_DCPERIOD      BETA  
0  43.538820 -92.85

In [25]:
print(data.isnull().sum())

date            0
open            0
high            0
low             0
close           0
volume          0
sma5            0
sma10           0
sma15           0
sma20           0
ema5            0
ema10           0
ema15           0
ema20           0
upperband       0
middleband      0
lowerband       0
HT_TRENDLINE    0
KAMA10          0
KAMA20          0
KAMA30          0
SAR             0
TRIMA5          0
TRIMA10         0
TRIMA20         0
ADX5            0
ADX10           0
ADX20           0
APO             0
CCI5            0
CCI10           0
CCI15           0
macd510         0
macd520         0
macd1020        0
macd1520        0
macd1226        0
MOM10           0
MOM15           0
MOM20           0
ROC5            0
ROC10           0
ROC20           0
PPO             0
RSI14           0
RSI8            0
slowk           0
slowd           0
fastk           0
fastd           0
fastksr         0
fastdsr         0
ULTOSC          0
WILLR           0
ATR             0
Trange    

In [26]:
data_zscore = data.select_dtypes(include=['float64', 'int64']).apply(zscore)
outliers = (data_zscore > 3) | (data_zscore < -3)
outliers_count = outliers.sum()
print("Initial outliers count:")
print(outliers_count)

Initial outliers count:
open             211
high             215
low              206
close            209
volume          1846
sma5             215
sma10            215
sma15            210
sma20            213
ema5             218
ema10            212
ema15            208
ema20            213
upperband        229
middleband       215
lowerband        199
HT_TRENDLINE     210
KAMA10           245
KAMA20           257
KAMA30           245
SAR                0
TRIMA5           214
TRIMA10          214
TRIMA20          210
ADX5             545
ADX10           1103
ADX20           1278
APO             2211
CCI5               0
CCI10            242
CCI15            591
macd510         2169
macd520         2232
macd1020        2271
macd1520        2301
macd1226        2309
MOM10           2130
MOM15           2178
MOM20           2196
ROC5            1969
ROC10           1970
ROC20           1921
PPO             1968
RSI14            326
RSI8              49
slowk              0
slowd     

In [27]:
import numpy as np

capped_data = data.copy()
for column in data.select_dtypes(include=['float64', 'int64']).columns:
    upper_limit = data[column].mean() + 3*data[column].std()
    lower_limit = data[column].mean() - 3*data[column].std()
    capped_data[column] = np.where(data[column] > upper_limit, upper_limit,
                                   np.where(data[column] < lower_limit, lower_limit, data[column]))

In [28]:
capped_data_zscore = capped_data.select_dtypes(include=['float64', 'int64']).apply(zscore)
rechecked_outliers = (capped_data_zscore > 3) | (capped_data_zscore < -3)
rechecked_outliers_count = rechecked_outliers.sum()
print("Outliers count after capping:")
print(rechecked_outliers_count)

Outliers count after capping:
open             217
high             218
low              208
close            214
volume          3850
sma5             215
sma10            220
sma15            212
sma20            216
ema5             218
ema10            212
ema15            210
ema20            214
upperband        229
middleband       215
lowerband        200
HT_TRENDLINE     211
KAMA10           250
KAMA20           263
KAMA30           247
SAR                0
TRIMA5           215
TRIMA10          218
TRIMA20          211
ADX5             560
ADX10           1179
ADX20           1417
APO             3411
CCI5               0
CCI10            244
CCI15            610
macd510         3423
macd520         3444
macd1020        3421
macd1520        3419
macd1226        3424
MOM10           3385
MOM15           3383
MOM20           3405
ROC5            3142
ROC10           2993
ROC20           2765
PPO             2964
RSI14            333
RSI8              49
slowk              0
slow

In [29]:
# Function to cap outliers using IQR
def cap_outliers_iqr(df):
    capped_df = df.copy()
    for column in df.select_dtypes(include=['float64', 'int64']).columns:
        Q1 = df[column].quantile(0.25)
        Q3 = df[column].quantile(0.75)
        IQR = Q3 - Q1
        lower_bound = Q1 - 1.5 * IQR
        upper_bound = Q3 + 1.5 * IQR
        capped_df[column] = np.where(df[column] > upper_bound, upper_bound,
                                     np.where(df[column] < lower_bound, lower_bound, df[column]))
    return capped_df

# Cap outliers using IQR
capped_data_iqr = cap_outliers_iqr(data)

# Recalculate Z-scores and recheck outliers
capped_data_iqr_zscore = capped_data_iqr.select_dtypes(include=['float64', 'int64']).apply(zscore)
rechecked_outliers_iqr = (capped_data_iqr_zscore > 3) | (capped_data_iqr_zscore < -3)
rechecked_outliers_count_iqr = rechecked_outliers_iqr.sum()
print("Outliers count after capping with IQR:")
print(rechecked_outliers_count_iqr)

Outliers count after capping with IQR:
open              0
high              0
low               0
close             0
volume            0
sma5              0
sma10             0
sma15             0
sma20             0
ema5              0
ema10             0
ema15             0
ema20             0
upperband         0
middleband        0
lowerband         0
HT_TRENDLINE      0
KAMA10            0
KAMA20            0
KAMA30            0
SAR               0
TRIMA5            0
TRIMA10           0
TRIMA20           0
ADX5              0
ADX10             0
ADX20             0
APO               0
CCI5              0
CCI10           243
CCI15             0
macd510           0
macd520           0
macd1020          0
macd1520          0
macd1226          0
MOM10             0
MOM15             0
MOM20             0
ROC5              0
ROC10             0
ROC20             0
PPO               0
RSI14             0
RSI8              0
slowk             0
slowd             0
fastk             0
f

In [30]:
# Check for duplicates before removing
duplicates_before = data.duplicated().sum()
print("Number of duplicate rows before removing:", duplicates_before)

Number of duplicate rows before removing: 0


In [31]:
# Feature Engineering: Extract date and time features
data['year'] = pd.to_datetime(data['date']).dt.year
data['month'] = pd.to_datetime(data['date']).dt.month
data['day'] = pd.to_datetime(data['date']).dt.day
data['hour'] = pd.to_datetime(data['date']).dt.hour

In [32]:
from sklearn.preprocessing import StandardScaler, MinMaxScaler, LabelEncoder
# Normalize and scale the features (excluding the date column)
scaler = StandardScaler()
data_scaled = scaler.fit_transform(capped_data_iqr.drop(columns=['date']))
data_scaled = pd.DataFrame(data_scaled, columns=capped_data_iqr.columns[1:])

In [33]:
# Compute the correlation matrix
correlation_matrix = data_scaled.corr().abs()

# Identify highly correlated features
upper_tri = correlation_matrix.where(np.triu(np.ones(correlation_matrix.shape), k=1).astype(bool))
highly_correlated_features = [column for column in upper_tri.columns if any(upper_tri[column] > 0.9)]

print("Highly correlated features (correlation > 0.9):")
for column in highly_correlated_features:
    print(f"{column}: {[col for col in upper_tri.columns if upper_tri.loc[column, col] > 0.9]}")

# Remove highly correlated features
data_reduced = data_scaled.drop(columns=highly_correlated_features)

# Final inspection
print("Data inspection after removing highly correlated features:")
print(data_reduced.head())
print(data_reduced.info())
print(data_reduced.describe())


Highly correlated features (correlation > 0.9):
high: ['low', 'close', 'sma5', 'sma10', 'sma15', 'sma20', 'ema5', 'ema10', 'ema15', 'ema20', 'upperband', 'middleband', 'lowerband', 'HT_TRENDLINE', 'KAMA10', 'KAMA20', 'KAMA30', 'TRIMA5', 'TRIMA10', 'TRIMA20', 'TYPPRICE']
low: ['close', 'sma5', 'sma10', 'sma15', 'sma20', 'ema5', 'ema10', 'ema15', 'ema20', 'upperband', 'middleband', 'lowerband', 'HT_TRENDLINE', 'KAMA10', 'KAMA20', 'KAMA30', 'TRIMA5', 'TRIMA10', 'TRIMA20', 'TYPPRICE']
close: ['sma5', 'sma10', 'sma15', 'sma20', 'ema5', 'ema10', 'ema15', 'ema20', 'upperband', 'middleband', 'lowerband', 'HT_TRENDLINE', 'KAMA10', 'KAMA20', 'KAMA30', 'TRIMA5', 'TRIMA10', 'TRIMA20', 'TYPPRICE']
sma5: ['sma10', 'sma15', 'sma20', 'ema5', 'ema10', 'ema15', 'ema20', 'upperband', 'middleband', 'lowerband', 'HT_TRENDLINE', 'KAMA10', 'KAMA20', 'KAMA30', 'TRIMA5', 'TRIMA10', 'TRIMA20', 'TYPPRICE']
sma10: ['sma15', 'sma20', 'ema5', 'ema10', 'ema15', 'ema20', 'upperband', 'middleband', 'lowerband', 'HT_TR